# Notebook RSNA - Bases

In [ ]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
import os
import pydicom

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from tqdm import tqdm
from collections import Counter
import math

## Première exploration des données

On commence par essayer d'analyser sous quelle forme sont les données.

On a accès a deux trains différents : 
- train.csv
- train_localizers.csv

Ces deux fichiers contiennent les informations (détails ci-après) nécessaires pour entraîner nos modèles aux deux tâches que l'on doit résoudre :
1. Déterminer s'il y a un anévrisme sur un fichier médical
2. Déterminer où se situe l'anévrisme parmis les différentes images accessibles (s'il y en a bien un)

In [ ]:
# Charger le CSV
train_df = pd.read_csv("/kaggle/input/rsna-intracranial-aneurysm-detection/train.csv")

# Aperçu : 5 premières lignes
print("Aperçu du train.csv :")
print(train_df.head())

# Nb total de séries
print("\nNombre total de séries :", len(train_df))

Vérification de la présence de valeurs manquantes :

In [ ]:
train_df.isna().sum()

Vérification de la présence de doublons :

In [ ]:
train_df['SeriesInstanceUID'].duplicated().sum()

### Train.csv

#### Structures des données
On a accès a **4348** séries (ensemble de slices DICOM provenant d'un même examen réalisé sur un même patient).

Les informations que l'on a sont :
- **SeriesInstanceUID** : identifiant unique de la série (sert à retrouver les DICOM dans le dossier series/).
- **PatientAge** : âge du patient (en années).
- **PatientSex** : sexe du patient (Male / Female).
- **Modality** : type d’imagerie (CTA, MRA, ou autre).
- **13 colonnes binaires** : indiquent la présence (1) ou absence (0) d’un anévrisme dans une localisation anatomique précise :
    - Left/Right Infraclinoid Internal Carotid Artery
    - Left/Right Supraclinoid Internal Carotid Artery
    - Left/Right Middle Cerebral Artery
    - Anterior Communicating Artery
    - Left/Right Anterior Cerebral Artery
    - Left/Right Posterior Communicating Artery
    - Basilar Tip
    - Other Posterior Circulation
- **Aneurysm Present** : cible principale (1 si au moins un anévrisme est détecté dans la série, 0 sinon).

Il est a noté qu'avec ces informations, l'on a pas moyen de savoir le nombre de patient distinct (plusieurs séries peuvent venir d'un patient). Ca peut être un biais important.

#### Âge des patients
Âge moyen : 58.5

Courbe en cloche (ou forme similaire)

In [ ]:
ages = train_df["PatientAge"]

# Stats
stats = ages.describe().round(1)
mean = stats["mean"]
median = stats["50%"]
std = stats["std"]

print("\nRésumé statistique :")
print(stats)

# Histogramme
plt.figure(figsize=(10,5))
plt.hist(ages, 
         bins=range(ages.min(), ages.max()+2), 
         edgecolor="black", align="left")

plt.title("Distribution des âges des patients")
plt.xlabel("Âge")
plt.ylabel("Nombre de séries")
plt.xticks(range(ages.min(), ages.max()+1, 2))

# Ajouter des lignes et annotations
plt.axvline(mean, color="red", linestyle="--", label=f"Moyenne = {mean:.1f}")
plt.axvline(median, color="green", linestyle="--", label=f"Médiane = {median:.1f}")
plt.axvline(mean-std, color="orange", linestyle=":", label=f"-1σ = {mean-std:.1f}")
plt.axvline(mean+std, color="orange", linestyle=":", label=f"+1σ = {mean+std:.1f}")

plt.legend()
plt.show()


#### Sexe des patients  
Répartition déséquilibré : presque deux fois plus de femme que d'hommes

Selon l'âge, la répartition hommes/femmes évolue mais reste globalement dans les mêmes tranches

In [ ]:
# Stats sur PatientSex
print("\n--- PatientSex ---")
print("Valeurs uniques :", train_df["PatientSex"].unique())
print("Counts :")
print(train_df["PatientSex"].value_counts())
print("\nPourcentages :")
print(train_df["PatientSex"].value_counts(normalize=True).round(3) * 100)

# Graphique avec seaborn
plt.figure(figsize=(5,5))
ax = sns.countplot(x="PatientSex", data=train_df, order=train_df["PatientSex"].value_counts().index)
plt.title("Répartition par sexe")
ax.bar_label(ax.containers[0])

plt.show()

In [ ]:
# Histogramme empilé par sexe
plt.figure(figsize=(12,6))
ax = sns.histplot(
    data=train_df,
    x="PatientAge",
    hue="PatientSex",
    bins=range(train_df["PatientAge"].min(), train_df["PatientAge"].max()+2),
    multiple="stack",
    edgecolor="black"
)

plt.title("Distribution des âges par sexe avec % de femmes")
plt.xlabel("Âge")
plt.ylabel("Nombre de séries")

# Calculer % de femmes par âge
age_sex_counts = train_df.groupby(["PatientAge", "PatientSex"]).size().unstack(fill_value=0)
age_sex_counts["Total"] = age_sex_counts.sum(axis=1)
age_sex_counts["Pct_Female"] = 100 * age_sex_counts.get("Female", 0) / age_sex_counts["Total"]

# Tracer la courbe du % femmes (échelle secondaire à droite)
ax2 = ax.twinx()
ax2.plot(age_sex_counts.index, age_sex_counts["Pct_Female"], color="red", marker="o", label="% Femmes")
ax2.set_ylabel("% Femmes")
ax2.set_ylim(0, 100)
ax2.legend(loc="upper right")

plt.show()

#### La modalité de la prise d'image

Plusieurs techniques d'imageries ont été utilisé dans cette base de donnée (ça va bien nous arranger dit donc) :
- MRA
- CTA
- MRI T2
- MRI T1post

Il faudra bien réfléchir à comment ont traite ces différentes techniques (qui sont bien déséquilibrés, pour ne rien arranger). Seul point positif, c'est relativement bien équilibré selon l'âge et le sexe.

In [ ]:
# Stats sur Modality
print("\n--- Modality ---")
print("Valeurs uniques :", train_df["Modality"].unique())
print("Counts :")
print(train_df["Modality"].value_counts())
print("\nPourcentages :")
print((train_df["Modality"].value_counts(normalize=True) * 100).round(1))

# Préparer les données
mod_counts = train_df["Modality"].value_counts().reset_index()
mod_counts.columns = ["Modality", "Count"]
mod_counts["Pourcentage"] = 100 * mod_counts["Count"] / len(train_df)

# Graphique
plt.figure(figsize=(7,5))
ax = plt.bar(mod_counts["Modality"], mod_counts["Count"], color="seagreen", edgecolor="black")

plt.title("Répartition par modalité d’imagerie")
plt.xlabel("Modality")
plt.ylabel("Nombre de séries")

# Ajouter les labels (count + %)
for i, (c, p) in enumerate(zip(mod_counts["Count"], mod_counts["Pourcentage"])):
    plt.text(i, c+20, f"{c}\n({p:.1f}%)", ha="center", va="bottom")

plt.show()

In [ ]:
# Moyenne, écart-type, count
stats = train_df.groupby(["Modality", "PatientSex"])["PatientAge"].agg(["mean", "std", "count"]).round(1)

# Ajouter les pourcentages par modalité
totals = stats.groupby(level=0)["count"].transform("sum")
stats["pct"] = (100 * stats["count"] / totals).round(1)

print("\n--- Âge par modality et sexe avec % ---")
print(stats)


#### Présence d'anévrisme
On a une présence d'anévrisme dans un peu moins de la moitié des séries.

Il y a une différence selon le sexe (47% chez les femmes contre 34% chez les hommes) et logiquement selon l'âge.

Pour les localisations spécifiques, elles diffèrent aussi pas mal selon l'âge et le sexe.

In [ ]:
# Stats globales
print("\n--- Aneurysm Present ---")
print("Counts :")
print(train_df["Aneurysm Present"].value_counts())
print("\nPourcentages :")
print((train_df["Aneurysm Present"].value_counts(normalize=True) * 100).round(2))

# Préparer les données
aneurysm_counts = train_df["Aneurysm Present"].value_counts().reset_index()
aneurysm_counts.columns = ["AneurysmPresent", "Count"]
aneurysm_counts["Pourcentage"] = 100 * aneurysm_counts["Count"] / len(train_df)

In [ ]:
# Stats par sexe et présence d’anévrisme
print("\n--- Aneurysm Present par sexe ---")
sex_counts = train_df.groupby(["PatientSex", "Aneurysm Present"]).size().unstack(fill_value=0)
sex_counts["Total"] = sex_counts.sum(axis=1)
sex_counts["Pct_Pos"] = (100 * sex_counts[1] / sex_counts["Total"]).round(2)

print(sex_counts)


In [ ]:
# Comptage par âge et statut d’anévrysme
age_counts = train_df.groupby(["PatientAge", "Aneurysm Present"]).size().unstack(fill_value=0)

ages = age_counts.index
neg = age_counts[0]  # sains
pos = age_counts[1]  # atteints

# Histogramme empilé
fig, ax1 = plt.subplots(figsize=(14,6))
ax1.bar(ages, neg, label="Non", color="skyblue", edgecolor="black")
ax1.bar(ages, pos, bottom=neg, label="Oui", color="salmon", edgecolor="black")

ax1.set_title("Présence d’anévrysme par âge avec % global")
ax1.set_xlabel("Âge")
ax1.set_ylabel("Nombre de séries")
ax1.legend(loc="upper left")
ax1.set_xticks(range(ages.min(), ages.max()+1, 2))

# % présence par âge
pct_present = 100 * pos / (pos + neg)
ax2 = ax1.twinx()
ax2.plot(ages, pct_present, color="black", marker="o", linewidth=1, label="% Présence (global)")
ax2.set_ylabel("% Présence")
ax2.set_ylim(0, 100)
ax2.legend(loc="upper right")

plt.show()


In [ ]:
# Colonnes de localisation (avec ID officiel)
loc_cols = [
    (1, "Other Posterior Circulation"),
    (2, "Basilar Tip"),
    (3, "Right Posterior Communicating Artery"),
    (4, "Left Posterior Communicating Artery"),
    (5, "Right Infraclinoid Internal Carotid Artery"),
    (6, "Left Infraclinoid Internal Carotid Artery"),
    (7, "Right Supraclinoid Internal Carotid Artery"),
    (8, "Left Supraclinoid Internal Carotid Artery"),
    (9, "Right Middle Cerebral Artery"),
    (10,"Left Middle Cerebral Artery"),
    (11,"Right Anterior Cerebral Artery"),
    (12,"Left Anterior Cerebral Artery"),
    (13,"Anterior Communicating Artery"),
]

# Totaux globaux
total_females = (train_df["PatientSex"] == "Female").sum()
total_males   = (train_df["PatientSex"] == "Male").sum()
total_aneurysms = train_df[[c for _, c in loc_cols]].sum().sum()

rows_summary = []
rows_stats = []

for loc_id, loc_name in loc_cols:
    subset = train_df[train_df[loc_name] == 1]
    count = len(subset)
    if count == 0:
        continue

    # --- Tableau 1 (résumé global) ---
    pct = 100 * count / total_aneurysms
    rows_summary.append([loc_id, loc_name, count, f"{pct:.1f}%"])

    # --- Tableau 2 (stats détaillées) ---
    mean_age = subset["PatientAge"].mean()
    std_age  = subset["PatientAge"].std()
    age_str  = f"{mean_age:.1f} ± {std_age:.1f}"

    # Femmes vs Hommes parmi les atteints
    n_f = (subset["PatientSex"] == "Female").sum()
    n_m = (subset["PatientSex"] == "Male").sum()

    # % Femmes parmi les atteints
    p_f = 100 * n_f / count
    sigma_f = np.sqrt((p_f/100) * (1 - p_f/100) / count) * 100

    # % Hommes parmi les atteints
    p_m = 100 * n_m / count
    sigma_m = np.sqrt((p_m/100) * (1 - p_m/100) / count) * 100

    # % Femmes atteintes rapporté au total femmes
    p_f_all = 100 * n_f / total_females
    sigma_f_all = np.sqrt((p_f_all/100) * (1 - p_f_all/100) / total_females) * 100

    # % Hommes atteints rapporté au total hommes
    p_m_all = 100 * n_m / total_males
    sigma_m_all = np.sqrt((p_m_all/100) * (1 - p_m_all/100) / total_males) * 100

    rows_stats.append([
        loc_id, age_str,
        f"{p_f:.1f}% ± {sigma_f:.1f}%", f"{p_m:.1f}% ± {sigma_m:.1f}%",
        f"{p_f_all:.2f}% ± {sigma_f_all:.2f}%", f"{p_m_all:.2f}% ± {sigma_m_all:.2f}%"
    ])

# --- DataFrames ---
summary_df = pd.DataFrame(rows_summary, columns=["ID", "Localisation", "N", "% parmi anévrismes"]).sort_values(by="N", ascending=False)
stats_df   = pd.DataFrame(rows_stats, columns=["ID", "Âge (moy ± σ)",
                                               "♀ / atteints", "♂ / atteints",
                                               "♀ atteintes (vs total ♀)",
                                               "♂ atteints (vs total ♂)"]).sort_values(by="ID")

# --- Affichage joli ---
print("\n--- Tableau 1 : Résumé global ---")
display(summary_df.style.set_properties(**{'text-align': 'center'}))

print("\n--- Tableau 2 : Statistiques détaillées ---")
display(stats_df.style.set_properties(**{'text-align': 'center'}))


### Multidimensional Scaling (MDS)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import MDS

# extract and scale input variables
input_MDS = train_df.drop(columns=['Aneurysm Present', 'SeriesInstanceUID'])
input_MDS.loc[:, 'PatientSex'] = input_MDS['PatientSex'].apply(lambda x: 1 if x == 'Female' or x == 1 else 0)
type_image_mapping = {'MRA': 0, 'CTA': 1, 'MRI T2': 2, 'MRI T1post': 3}
input_MDS.loc[:, 'Modality'] = input_MDS['Modality'].map(type_image_mapping)
input_MDS_scale = StandardScaler().fit_transform(input_MDS)
# perform MDS (2D)
transform_MDS = MDS(n_components=2, random_state=0, dissimilarity='euclidean')
coords_MDS = transform_MDS.fit_transform(input_MDS_scale)

In [ ]:
# plot projection
plt.figure(figsize=(8,6))
plt.scatter(coords_MDS[:,0],
            coords_MDS[:,1],
            c=train_df['Aneurysm Present'],
            cmap='coolwarm')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('2D Projection on Clinical Data')
plt.colorbar(label='Aneurysm (1 = yes , 0 = no)')
plt.show()

### series/

Chaque series contient de nombreux fichier .dcm.

On va notamment regarder combien il y en a et sous quelle forme les fichiers dcm sont présentés.

Il y a un gros problème de normalisation : les séries n'ont pas le même nombre de DICOM, les tailles des images ne sont pas similaires, les prises d'images n'ont pas les mêmes point de départ et d'arrivé, certaines images sont en 2d tandis que d'autres sont en 3d.

In [ ]:
# Dossier racine
data_dir = "/kaggle/input/rsna-intracranial-aneurysm-detection/series/"

# --------------------
# 1. Analyse des séries
# --------------------
series_counts = []
for series_uid in tqdm(os.listdir(data_dir), desc="Analyse des séries"):
    series_path = os.path.join(data_dir, series_uid)
    if os.path.isdir(series_path):
        dcm_files = [f for f in os.listdir(series_path) if f.endswith(".dcm")]
        n_dcm = len(dcm_files)
        series_counts.append((series_uid, n_dcm))

# --------------------
# 2. Créer DataFrame
# --------------------
df_series = pd.DataFrame(series_counts, columns=["SeriesInstanceUID", "n_dcm"])

# --------------------
# 3. Stats globales
# --------------------
print("\nStatistiques sur le nombre de DICOM par série :")
print(df_series["n_dcm"].describe())

# --------------------
# 4. Top 5 min / max / médiane
# --------------------
# Top 5 moins peuplées
top5_small = df_series.nsmallest(5, "n_dcm")

# Top 5 plus peuplées
top5_large = df_series.nlargest(5, "n_dcm")

# Top 5 proches de la médiane
median_val = df_series["n_dcm"].median()
df_series["dist_to_median"] = (df_series["n_dcm"] - median_val).abs()
top5_median = df_series.nsmallest(5, "dist_to_median")

print("\n=== Top 5 séries les moins peuplées ===")
print(top5_small[["SeriesInstanceUID", "n_dcm"]])

print("\n=== Top 5 séries les plus peuplées ===")
print(top5_large[["SeriesInstanceUID", "n_dcm"]])

print(f"\nMédiane du nombre de DICOM : {median_val}")
print("\n=== Top 5 séries proches de la médiane ===")
print(top5_median[["SeriesInstanceUID", "n_dcm"]])

In [ ]:
def show_dicom(series_uid, data_dir, index=0):
    series_path = os.path.join(data_dir, series_uid)
    dcm_files = sorted([f for f in os.listdir(series_path) if f.endswith(".dcm")])
    if not dcm_files:
        print(f"Aucun fichier DICOM trouvé dans {series_uid}")
        return
    
    if index >= len(dcm_files):
        print(f"Index {index} hors limites (la série contient {len(dcm_files)} DICOMs)")
        return
    
    dicom_path = os.path.join(series_path, dcm_files[index])
    ds = pydicom.dcmread(dicom_path)
    print(ds)

In [ ]:
show_dicom(df_series.iloc[0]["SeriesInstanceUID"], data_dir)

In [ ]:
import os
import pydicom
import pandas as pd
from tqdm import tqdm

def check_body_parts(data_dir):
    results = []
    for series_uid in tqdm(os.listdir(data_dir), desc="Scan séries"):
        series_path = os.path.join(data_dir, series_uid)
        if not os.path.isdir(series_path):
            continue

        # prendre le premier DICOM
        dcm_files = [f for f in os.listdir(series_path) if f.endswith(".dcm")]
        if not dcm_files:
            continue

        dcm_path = os.path.join(series_path, sorted(dcm_files)[0])
        try:
            ds = pydicom.dcmread(dcm_path, stop_before_pixels=True)  # rapide, pas besoin des pixels
            body_part = getattr(ds, "BodyPartExamined", "UNKNOWN")
        except Exception as e:
            body_part = f"ERROR: {e}"

        results.append((series_uid, body_part))

    return pd.DataFrame(results, columns=["SeriesInstanceUID", "BodyPartExamined"])

# Exemple d’utilisation
df_body = check_body_parts(data_dir)
print(df_body["BodyPartExamined"].value_counts())

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pydicom

def show_series(series_idx_list, df_series, data_dir, cols=8):
    """
    Affiche plusieurs séries DICOM à partir de leur indice dans df_series.
    
    series_idx_list : liste d'indices (int) de séries dans df_series
    df_series       : DataFrame contenant au moins une colonne 'SeriesInstanceUID'
    data_dir        : dossier racine contenant les séries
    cols            : nombre d'images par ligne
    """
    for idx in series_idx_list:
        series_uid = df_series.iloc[idx]["SeriesInstanceUID"]
        series_path = os.path.join(data_dir, series_uid)
        dcm_files = [f for f in os.listdir(series_path) if f.endswith(".dcm")]

        images, metas = [], []
        for fname in dcm_files:
            dcm_path = os.path.join(series_path, fname)
            ds = pydicom.dcmread(dcm_path)
            arr = ds.pixel_array

            # Cas multi-frame : plusieurs slices dans un seul DICOM
            if arr.ndim == 3:
                for slice_idx in range(arr.shape[0]):
                    images.append(arr[slice_idx])
                    metas.append({
                        "file": f"{fname}[{slice_idx}]",
                        "instance": getattr(ds, "InstanceNumber", None),
                        "slice_loc": getattr(ds, "SliceLocation", None),
                        "shape": arr[slice_idx].shape,
                        "slice_thickness": getattr(ds, "SliceThickness", None),
                        "pixel_spacing": getattr(ds, "PixelSpacing", None),
                    })
            else:
                images.append(arr)
                metas.append({
                    "file": fname,
                    "instance": getattr(ds, "InstanceNumber", None),
                    "slice_loc": getattr(ds, "SliceLocation", None),
                    "shape": arr.shape,
                    "slice_thickness": getattr(ds, "SliceThickness", None),
                    "pixel_spacing": getattr(ds, "PixelSpacing", None),
                })

        # Trier par InstanceNumber (si dispo)
        paired = list(zip(images, metas))
        paired.sort(key=lambda x: x[1]["instance"] if x[1]["instance"] is not None else 0)
        images, metas = zip(*paired)
        images, metas = list(images), list(metas)

        # Volume 3D (si plusieurs slices)
        volume = np.stack(images, axis=-1) if len(images) > 1 else images[0]

        # Stats
        print(f"\n=== Série {series_uid} (index {idx}) ===")
        print(f"Nombre d'images (après expansion multi-frame) : {len(images)}")
        print(f"Forme du volume : {np.array(images).shape}")
        print("Épaisseur de coupe :", metas[0]["slice_thickness"])
        print("PixelSpacing :", metas[0]["pixel_spacing"])
        print("InstanceNumber min/max :", 
              min(m['instance'] for m in metas if m['instance'] is not None), 
              "/", 
              max(m['instance'] for m in metas if m['instance'] is not None))

        # Affichage
        n = len(images)
        rows = int(np.ceil(n / cols))
        fig, axes = plt.subplots(rows, cols, figsize=(20, 2.5*rows))
        axes = axes.flatten()

        for i, img in enumerate(images):
            axes[i].imshow(img, cmap="gray")
            axes[i].set_title(f"{metas[i]['file']}")
            axes[i].axis("off")

        for j in range(i+1, len(axes)):
            axes[j].axis("off")

        plt.suptitle(f"Série {series_uid} (index {idx}) : {n} images", fontsize=16)
        plt.tight_layout()
        plt.show()


In [ ]:
show_series([120, 163], df_series, data_dir)

In [ ]:
show_series([2250, 3081], df_series, data_dir)

In [ ]:
show_series([12, 33], df_series, data_dir)

In [ ]:
data_dir = "/kaggle/input/rsna-intracranial-aneurysm-detection/series/"

sizes = []

# Lire uniquement 1 DICOM par série
for series_uid in tqdm(os.listdir(data_dir), desc="Analyse des séries"):
    series_path = os.path.join(data_dir, series_uid)
    if os.path.isdir(series_path):
        dcm_files = [os.path.join(series_path, f) for f in os.listdir(series_path) if f.endswith(".dcm")]
        if not dcm_files:
            continue
        try:
            ds = pydicom.dcmread(dcm_files[0], stop_before_pixels=True)
            sizes.append((ds.Rows, ds.Columns))
        except Exception as e:
            print(f"⚠️ Erreur lecture {dcm_files[0]}: {e}")

# Compter les tailles
counts = Counter(sizes)

# Trier par fréquence décroissante
sorted_sizes = sorted(counts.items(), key=lambda x: x[1], reverse=True)

print("\n📏 Tailles d’images rencontrées (triées par fréquence) :")
for (rows, cols), n in sorted_sizes:
    g = math.gcd(rows, cols)
    ratio = f"{cols//g}:{rows//g}"  # largeur:hauteur
    print(f" - {rows} x {cols}  (ratio {ratio}) → {n} séries")

# Stats globales
total_series = sum(counts.values())
n_unique_sizes = len(counts)
most_common_size, most_common_count = sorted_sizes[0]

print("\n📊 Statistiques globales :")
print(f"Nombre total de séries analysées : {total_series}")
print(f"Nombre de tailles uniques : {n_unique_sizes}")
print(f"Taille la plus fréquente : {most_common_size} → {most_common_count} séries "
      f"({100*most_common_count/total_series:.1f}%)")
